In [1]:
import sys

sys.path.append("..")

from pathlib import Path
from tqdm import tqdm
from PIL import Image
from drone.action_aviation_multi_camera import ActionAviationMultiCameraFlight

In [ ]:
FLIGHT_DIR = Path("/workspaces/cv/data/surveys/action-aviation-multicamera/20250405_west_coast_4camera/flight_1/")
DATA_DIR = Path("/workspaces/cv/survey-reviewer/data")
CROP_H = 240

In [3]:
import concurrent.futures

# First, thumbnail everything and get the metadata:
flight = ActionAviationMultiCameraFlight(FLIGHT_DIR)

thumb_dir = DATA_DIR / "thumbnails"
thumb_dir.mkdir(exist_ok=True, parents=True)
img_meta = []


def do(img_path, thumb_path):
    if not thumb_path.exists():
        # Create a thumbnail of the image
        img = Image.open(img_path)
        h, w = img.height, img.width
        crop_h = CROP_H
        crop_w = int(CROP_H / h * w)
        img.thumbnail((crop_w, crop_h))
        # if photo.camera_name.lower() in ("r28", "l09"):
        #     # These cameras are mounted backwards, so rotate 180 degrees:
        #     img = img.rotate(180)
        # if photo.camera_name.lower() in ("l28", "l09", "l28"):
        #     # These cameras are mounted backwards, so rotate 180 degrees:
        #     img = img.rotate(180)
        img.save(thumb_path)


photos = list(sorted(tqdm(flight.get_located_photos()), key=lambda x: x.time))

for photo in photos:
    thumb_path = thumb_dir / f"{photo.idx:06d}_{photo.camera_name}.jpg"
    lat, lon = photo.location.lat, photo.location.lon
    img_meta.append(
        dict(
            idx=photo.idx,
            camera=photo.camera_name,
            name=photo.path.name,
            time=photo.time,
            location=photo.location,
            thumbnail_name=thumb_path.name,
        )
    )

# Create all the thumbs:
with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    futures = []
    for photo in photos:
        thumb_path = thumb_dir / f"{photo.idx:06d}_{photo.camera_name}.jpg"
        futures.append(executor.submit(do, photo.path, thumb_path))
    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures)):
        try:
            future.result()
        except Exception as e:
            print(f"Error processing thumbnail: {e}")
            continue

    # break
img_meta[0]


14952it [00:00, 3063066.98it/s]
100%|██████████| 14952/14952 [40:31<00:00,  6.15it/s] 


{'idx': 0,
 'camera': 'R28',
 'name': '_28R3139.JPG',
 'time': <Arrow [2025-04-05T13:18:29.012000+13:00]>,
 'location': Location(lat=-37.761752452, lon=174.824964298, altitude=478.7575, time=None, image_name=None),
 'thumbnail_name': '000000_R28.jpg'}

In [5]:
from copy import deepcopy
import pyproj

geodesic = pyproj.Geod(ellps="WGS84")

# OK, project them. Per camera, get the GPS track
unique_cameras = list(set([i["camera"] for i in img_meta]))
# TODO: calculate offsets from altitude
offsets = {
    "L28": (-550, -250),
    "L09": (-250, 0),
    "R09": (0, 250),
    "R28": (250, 550),
}
image_height = 200
filtered_imgs = []
for c in unique_cameras:
    # print(c)
    meta = [i for i in img_meta if i["camera"] == c]
    meta = sorted(meta, key=lambda x: x["time"])
    x0, x1 = offsets[c]
    for i in range(len(meta)):
        # for i in range(10):
        if i == 0:
            p0 = meta[i]["location"]
            p1 = meta[i + 1]["location"]
        else:
            p0 = meta[i - 1]["location"]
            p1 = meta[i]["location"]
        bearing, _, _ = geodesic.inv(p0.lon, p0.lat, p1.lon, p1.lat)
        # OK, cool. Let's project each image
        m = meta[i]
        p = m["location"]
        forward_lon, forward_lat, _ = geodesic.fwd(p.lon, p.lat, bearing, image_height / 2)
        rear_lon, rear_lat, _ = geodesic.fwd(p.lon, p.lat, bearing + 180, image_height / 2)
        angle = bearing - 90 if x0 < 0 else bearing + 90
        top_left_lon, top_left_lat, _ = geodesic.fwd(forward_lon, forward_lat, angle, abs(x0))
        top_right_lon, top_right_lat, _ = geodesic.fwd(forward_lon, forward_lat, angle, abs(x1))
        bottom_left_lon, bottom_left_lat, _ = geodesic.fwd(rear_lon, rear_lat, angle, abs(x0))
        bottom_right_lon, bottom_right_lat, _ = geodesic.fwd(rear_lon, rear_lat, angle, abs(x1))

        # print(p.lon, p.lat, left_lon, bottom_lat, right_lon, top_lat)
        m = deepcopy(m)
        m["bounds_lbrt"] = [
            [bottom_left_lon, bottom_left_lat],
            [top_left_lon, top_left_lat],
            [top_right_lon, top_right_lat],
            [bottom_right_lon, bottom_right_lat],
        ]
        filtered_imgs.append(m)
        # s = [f"{lon} {lat}" for lon, lat in m["bounds_lbrt"]]
        # s = ", ".join(s)
        # print(f"LINESTRING({s})")

print(len(filtered_imgs))
filtered_imgs[0]

14952


{'idx': 1,
 'camera': 'L28',
 'name': '_28L0013.JPG',
 'time': <Arrow [2025-04-05T13:18:29.012000+13:00]>,
 'location': Location(lat=-37.761752452, lon=174.824964298, altitude=478.7575, time=None, image_name=None),
 'thumbnail_name': '000001_L28.jpg',
 'bounds_lbrt': [[174.81886150325502, -37.763128512489516],
  [174.81864202871333, -37.76133503602259],
  [174.82203065229197, -37.76107362801284],
  [174.82225020865576, -37.76286710456792]]}

In [6]:
import json

# Save it out:
out = []
n = max(i["idx"] for i in filtered_imgs)
for i in filtered_imgs:
    ppn = i["idx"] / n
    out.append(
        dict(
            camera=i["camera"],
            name=i["name"],
            time=i["time"].strftime("%Y-%m-%d %H:%M:%S"),
            flight_ppn=ppn,
            location=dict(
                lat=round(i["location"].lat, 6), lon=round(i["location"].lon, 6), alt=int(i["location"].altitude)
            ),
            thumbnail_name=i["thumbnail_name"],
            bounds_lbrt=[[round(lon, 6), round(lat, 6)] for lon, lat in i["bounds_lbrt"]],
        )
    )
with open(DATA_DIR / "imgs.json", "w") as f:
    json.dump(out, f)

In [7]:
import struct

# Encode the images:
obites = []
for i in filtered_imgs:
    with open(thumb_dir / i["thumbnail_name"], "rb") as f:
        bites = f.read()
    obites.append(struct.pack(">II", len(i["thumbnail_name"]), len(bites)))
    obites.append(i["thumbnail_name"].encode("utf-8"))
    obites.append(bites)
with open(DATA_DIR / "imgs.bin", "wb") as f:
    for i in obites:
        if isinstance(i, bytes):
            f.write(i)
        else:
            f.write(i[0:4])